When you import Sweatpy with `import sweat` all pandas data frames and series have a `sweat` accessor available that allows you to do things like:
```python
df.sweat.mean_max("column name")
```

...or

```python
df["power"].sweat.mean_max()
```

See below for more examples.
The sweat accessor will raise an `AttributeError` when the data frame or series content is not valid.

Please note that the sweat accessor for data frames and series are *similar* but not *identical*.

Most methods on the sweat accessor are also available for usage outside data frames.

## Mean max

In [1]:
import sweat

example = sweat.examples(path="4078723797.fit")
data = sweat.read_fit(example.path)

mmp = data["power"].sweat.mean_max()
mmp

0 days 00:00:01    1022.000000
0 days 00:00:02     980.500000
0 days 00:00:03     928.666667
0 days 00:00:04     847.750000
0 days 00:00:05     721.600000
                      ...     
0 days 02:19:12     219.201748
0 days 02:19:13     219.175506
0 days 02:19:14     219.149270
0 days 02:19:15     219.123040
0 days 02:19:16     219.096817
Name: mean_max_power, Length: 8356, dtype: float64

Or for multiple columns at once:

In [2]:
import sweat

example = sweat.examples(path="4078723797.fit")
data = sweat.read_fit(example.path)

mean_max = data.sweat.mean_max(["power", "heartrate"])
mean_max

,mean_max_power,mean_max_heartrate
0 days 00:00:01,1022.000000,190.000000
0 days 00:00:02,980.500000,190.000000
0 days 00:00:03,928.666667,190.000000
0 days 00:00:04,847.750000,190.000000
0 days 00:00:05,721.600000,190.000000
...,...,...
0 days 02:19:12,219.201748,164.746049
0 days 02:19:13,219.175506,164.738178
0 days 02:19:14,219.149270,164.730668
0 days 02:19:15,219.123040,164.723639


The `mean_max()` methods accept a `monotic` boolean argument that can be used to force a monotonically decreasing mean max curve. The default is `False`.

In [1]:
import sweat

example = sweat.examples(path="4078723797.fit")
data = sweat.read_fit(example.path)

mean_max = data.sweat.mean_max(["power", "heartrate"], monotonic=True)

The `mean_max()` function is also available as `sweat.metrics.core.mean_max()`.

## Timedelta index

The data frames returned by the `read_*()` functions have a [pandas.DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) by default.
Sometimes it is usefull to have a relative [pandas.TimedeltaIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.TimedeltaIndex.html):

In [4]:
import sweat

example = sweat.examples(path="4078723797.fit")
activity = sweat.read_fit(example.path)[["power", "heartrate"]]

activity = activity.sweat.to_timedelta_index()
activity.head()

,power,heartrate
datetime,,
00:00:00,0,111
00:00:01,0,108
00:00:02,0,106
00:00:03,0,102
00:00:04,0,99


Then you can do things like slice for the first 10 seconds of an activity:

In [5]:
activity[:"00:00:10"]

,power,heartrate
datetime,,
00:00:00,0,111
00:00:01,0,108
00:00:02,0,106
00:00:03,0,102
00:00:04,0,99
00:00:05,0,95
00:00:06,0,94
00:00:07,0,94
00:00:08,0,97


The `sweat.to_timedelta_index()` method is available on both data frames and series.

## Training zones

Working with training zones is easy in Sweatpy. To add a column with the heart rate zone label to the data frame:

In [5]:
import sweat

example = sweat.examples(path="4078723797.fit")
activity = sweat.read_fit(example.path)[["power", "heartrate"]]

activity["heartrate_zone"] = activity["heartrate"].sweat.calculate_zones(
    bins=[0, 100, 140, 160, 999],
    labels=["rest", "D1", "D2", "D3"])
activity["heartrate_zone"].head()

datetime
2019-09-20 14:10:03+00:00      D1
2019-09-20 14:10:04+00:00      D1
2019-09-20 14:10:05+00:00      D1
2019-09-20 14:10:06+00:00      D1
2019-09-20 14:10:07+00:00    rest
Name: heartrate_zone, dtype: category
Categories (4, object): [rest < D1 < D2 < D3]

...where the `bins` argument contains the left and right bounds for each training zone and the `labels` argument the zone labels.

To calculate the time in zone:

In [11]:
import sweat

example = sweat.examples(path="4078723797.fit")
activity = sweat.read_fit(example.path)[["power", "heartrate"]]

time_in_zone = activity["power"].sweat.time_in_zone(
    bins=[-9999, 150, 230, 320, 9999],
    labels=["rest", "D1", "D2", "D3"])
time_in_zone

D2     00:58:06
rest   00:36:25
D3     00:25:40
D1     00:19:06
Name: power, dtype: timedelta64[ns]